In [1]:
!pip install transformers sentence-transformers -Uq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 31.3 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import cdist
from typing import List, Tuple
import torch

# Initialize the language model and tokenizer
model_name = "Qwen/Qwen2.5-1.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Initialize the embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
# Sample set of documents.
corpus = [
    "ICTer 2024 at UCSC.",
    "LLM workshop is from 1 pm to 4pm by Keshan Sodimana",
    "Artificial Intelligence is transforming the world.",
    "5th of Novemeber is the first day of ICTer 2024.",
    "Python is a popular programming language for data science."
]

# Generate embeddings for the corpus and store in a DataFrame (vector store)
corpus_embeddings = embedding_model.encode(corpus)
df = pd.DataFrame({"text": corpus, "embedding": list(corpus_embeddings)})


In [12]:
# Function to retrieve top-k most similar documents
def retrieve(query: str, k: int = 3) -> List[Tuple[str, float]]:
    query_embedding = embedding_model.encode([query])
    query_embedding = np.array(query_embedding).reshape(1, -1)
    embeddings_matrix = np.vstack(df['embedding'].to_list())
    similarities = 1 - cdist(query_embedding, embeddings_matrix, metric="cosine")[0]
    top_k_idx = np.argsort(similarities)[-k:][::-1]

    return [(df.iloc[idx]['text'], similarities[idx]) for idx in top_k_idx]


# Function to generate response with RAG
def generate_response(query: str) -> str:
    retrieved_docs = retrieve(query)
    context = " ".join([doc[0] for doc in retrieved_docs])
    prompt = f"{context}\n\nQuery: {query}\nAnswer:"

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)


    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_length=100)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [20]:
query = "Where is the ICTer LLM workshop"
response = generate_response(query)
print(response.split('Answer:')[-1])

 The ICTer LLM workshop is held on November 5th, and it starts at 1 PM and ends at 4 PM. The location for this event is at UC
